#### Project: A convolutional Neural Networks with all layers purely convolution layers

#### Packages selection
- The first things is to import all the neccesary packages needed for this project

In [19]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader


# selct GPU if cuda is available
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

#### Settings
- Configure the device
- define all the hyperparameters to be used and needed to be tuned to achive a better accuracy
- Load and explore the data

In [20]:
# device
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

# hyperparameters
random_seed = 1
learning_rate = 0.001
num_epochs = 15
batch_size = 256

# model architecture parameters
num_classes = 10

# dataset -> MNIST
# Note: transforms.ToTensor() scale image from 0-1 range

train_dataset = datasets.MNIST(root='data',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

test_dataset = datasets.MNIST(root='data',
                             train=False,
                             transform=transforms.ToTensor())

train_loader = DataLoader(dataset=train_dataset,
                         batch_size=batch_size,
                         shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                        batch_size=batch_size,
                        shuffle=False)

# check the dataset
for images, labels in train_loader:
    print("Image Batch dimension", images.shape)
    print("Image Labels dimension", labels.shape)
    break

Image Batch dimension torch.Size([256, 1, 28, 28])
Image Labels dimension torch.Size([256])


#### Define the architecture of the model such as
- The number of input layers; which is determined by the features of the data
- Number of total hidden layers in the model consist of only Convolution layers
- The output layer node units is determined by the intended outcome to achieve
- Here we have two layers convolutional neural networks

In [21]:
"""
Architecture
X -> Conv -> ReLU -> Conv -> ReLU -> Conv -> ReLU -> Conv -> ReLU -> Conv -> ReLU -> Conv -> ReLU -> Conv -> Softmax -> y
"""

class ConvNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        self.num_classes = num_classes
        
        self.conv_1 = nn.Conv2d(in_channels=1,
                               out_channels=4,
                               kernel_size=(3, 3),
                               stride=(1, 1),
                               padding=1)
        
        self.conv_2 = nn.Conv2d(in_channels=4,
                               out_channels=4,
                               kernel_size=(3, 3),
                               stride=(2, 2),
                               padding=1)
        
        self.conv_3 = nn.Conv2d(in_channels=4,
                               out_channels=8,
                               kernel_size=(3, 3),
                               stride=(1, 1),
                               padding=1)
        
        self.conv_4 = nn.Conv2d(in_channels=8,
                               out_channels=8,
                               kernel_size=(3, 3),
                               stride=(2, 2),
                               padding=1)
        
        self.conv_5 = nn.Conv2d(in_channels=8,
                               out_channels=16,
                               kernel_size=(3, 3),
                               stride=(1, 1),
                               padding=1)
        
        self.conv_6 = nn.Conv2d(in_channels=16,
                               out_channels=16,
                               kernel_size=(3, 3),
                               stride=(2, 2),
                               padding=1)
        
        self.conv_7 = nn.Conv2d(in_channels=16,
                               out_channels=self.num_classes,
                               kernel_size=(3, 3),
                               stride=(1, 1),
                               padding=1)
        
    def forward(self, x):
        out = self.conv_1(x)
        out = F.relu(out)
        
        out = self.conv_2(out)
        out = F.relu(out)
        
        out = self.conv_3(out)
        out = F.relu(out)
        
        out = self.conv_4(out)
        out = F.relu(out)
        
        out = self.conv_5(out)
        out = F.relu(out)
        
        out = self.conv_6(out)
        out = F.relu(out)
        
        out = self.conv_7(out)
        out = F.relu(out)
        
        logits = F.adaptive_avg_pool2d(out, 1)
        # drop the width
        logits.squeeze_(-1)
        # drop the height
        logits.squeeze_(-1)
        probas = torch.softmax(logits, dim=1)
        return logits, probas

#### Loss function and optimizer
- Instantiate the model
- define the specific Loss function to be used either cross entropy, MSELoss, etc
- define the optimization algorithm to be used either SGD, Adam, RMSprop, Momentum etc.

In [22]:
torch.manual_seed(random_seed)
model = ConvNet(num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#### compute accuracy
- A function to compute train and test accuracy

In [23]:
def compute_accuracy(model, data_loader):
    correct_predictions, num_examples = 0, 0
    for features, labels in data_loader:
        features = features.to(device)
        labels = labels.to(device)
        outputs, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += labels.size(0)
        correct_predictions += (predicted_labels == labels).sum()
    return correct_predictions.float() / num_examples * 100

#### Training a model requires the following steps
- Reset all the gradients to zero (0)
- Make a forward pass (make a prediction)
- Calculate the loss
- Perform back propagation
- Update all the parameters (weight and biases)

In [24]:
start_time = time.time()
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):
        features = features.to(device)
        labels = labels.to(device)
        
        # Forwward and back prop
        output, probas = model(features)
        loss = F.cross_entropy(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Logging
        if not i % 50:
            print('Epoch: %03d/%03d | Batch: %03d/%03d | Cost: %.4f'
                 %(epoch+1, num_epochs, i, total_step, loss))
        
    model = model.eval()
    print('Epoch: %03d/%03d training accuracy: %.2f%%' %(
    epoch+1, num_epochs, compute_accuracy(model, train_loader)))
    print('Time Elapsed: %.2f min' %((time.time() - start_time) / 60))
print('Total Training time: %.2f min' %((time.time() - start_time) / 60))

Epoch: 001/015 | Batch: 000/235 | Cost: 2.3051
Epoch: 001/015 | Batch: 050/235 | Cost: 2.2912
Epoch: 001/015 | Batch: 100/235 | Cost: 2.0543
Epoch: 001/015 | Batch: 150/235 | Cost: 1.5351
Epoch: 001/015 | Batch: 200/235 | Cost: 1.1741
Epoch: 001/015 training accuracy: 74.54%
Time Elapsed: 1.12 min
Epoch: 002/015 | Batch: 000/235 | Cost: 1.0367
Epoch: 002/015 | Batch: 050/235 | Cost: 0.8080
Epoch: 002/015 | Batch: 100/235 | Cost: 0.6701
Epoch: 002/015 | Batch: 150/235 | Cost: 0.4320
Epoch: 002/015 | Batch: 200/235 | Cost: 0.6095
Epoch: 002/015 training accuracy: 82.90%
Time Elapsed: 2.25 min
Epoch: 003/015 | Batch: 000/235 | Cost: 0.5706
Epoch: 003/015 | Batch: 050/235 | Cost: 0.4780
Epoch: 003/015 | Batch: 100/235 | Cost: 0.5257
Epoch: 003/015 | Batch: 150/235 | Cost: 0.5403
Epoch: 003/015 | Batch: 200/235 | Cost: 0.6229
Epoch: 003/015 training accuracy: 84.66%
Time Elapsed: 3.43 min
Epoch: 004/015 | Batch: 000/235 | Cost: 0.5428
Epoch: 004/015 | Batch: 050/235 | Cost: 0.5109
Epoch: 00

#### Evaluate

In [25]:
# Print the accuracy
print("Test Accuracy:  %.2f%%" %(compute_accuracy(model, test_loader)))

Test Accuracy:  88.38%
